- https://pjt3591oo.github.io/
- 소스보기를 통해 서버로 부터 받아오는 소스에 데이터가 있는 것이 확인된 경우
- requests 모듈을 통해 요청한다.
- bs4를 통해 분석해서 수집한다.

In [1]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os
from IPython.display import clear_output

In [2]:
# 요청 함수
def getSource(site) :
    # 해더 정보 셋팅
    # user-agent : 웹브라우저가 서버로 보내는 문자열이고 서버는 이를 통해
    # 브라우저의 정보가 컴퓨터의 정보를 파악한다.
    # 일부 사이트는 user-agent를 셋팅해서 지금 요청한 도구가 python코드가 아닌 일반 웹 브라우저 인것 처럼 속일 수 있다.
    # http://m.avalon.co.kr/check.html
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    # 요청한다.
    response = requests.get(site, headers=header_info)
    return response.text

In [3]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(source):
    
    # bs4 객체를 생성한다.
    soup = bs4.BeautifulSoup(source)
    
    # 데이터가 있는 전체를 가져온다.
    root = soup.select('body > main > div > div > div')
    # for a1 in root :
        # print(a1)
        # print('----------------------------')
    
    # 데이터 프레임을 생성하기 위해 사용할 딕셔너리
    data_dict = {
        '큰제목' : [],
        '작은제목' : [],
        '날짜' : [],
        '작성자' : []
    }
    
    # 가져온 태그의 수만큼 반복한다.
    for div_tag in root[:-1] :
        # 큰 제목을 가져온다.
        a1 = div_tag.select_one('h3 > a')
        data1 = getText(a1)
        a2 = div_tag.select_one('h4')
        data2 = getText(a2)
        a3 = div_tag.select_one('p > span')
        a4 = getText(a3)
        a5 = a4.split('|')
        data3 = a5[0].strip()
        data4 = a5[1].strip()
        
        # 데이터를 담는다.
        data_dict['큰제목'].append(data1)
        data_dict['작은제목'].append(data2)
        data_dict['날짜'].append(data3)
        data_dict['작성자'].append(data4)

        # print(data1)
        # print(data2)
        # print(data3)
        # print(data4)
        # print('----------------------------')
        
    # 데이터 프레임 생성
    df1 = pd.DataFrame(data_dict)
    # display(df1)
    
    # 저장
    # 파일이 존재하지 않은다면, 컬럼이름과 함께 저장한다.
    if os.path.exists('./02_data.csv') == False :
        df1.to_csv('./02_data.csv', encoding='utf-8-sig', index=False)
    else :
        df1.to_csv('./02_data.csv', encoding='utf-8-sig', index=False, header=None, mode='a') #  mode='a' : 파일 뒤에 이어서 저장

In [4]:
# 전달된 태그로 부터 문자열 데이터를 추출해 반환하는 함수
def getText(tag):
    # 태그가 없다면
    if tag == None :
        return np.nan
    # 문자열을 가져온다.
    data = tag.text.strip()
    # 길이가 0 이라면...
    if len(data) == 0 :
        return np.nan
    return data

In [5]:
# 마지막 페이지인지 확인하는 함수
def check_last_page(source):
    # bs4 객체를 생성한다.
    soup = bs4.BeautifulSoup(source)
    
    # Next 버튼 태그를 가져온다.
    a1 = soup.select_one('body > main > div > div > div.pagination > ul > li:nth-child(6) > a')
    # 가져온 태그가 없다면 마지막 페이지가지 간 것으로 간주한다.
    if a1 == None : 
        return True

In [6]:
# 페이지의 주소
site = 'https://pjt3591oo.github.io'
page = ''
page_num = 1

while True:
    # 딜레이
    time.sleep(1)
    
    # 기존에 출력된 것을 지운다.
    clear_output(wait=True)
    
    print(f'{site}{page} 수집중...')
    
    # 현재 요청 주소 기록
    # 첫 페이지면 덮어 씌운다.
    if page == '' :
        mode = 'wt'
    else:
        mode = 'at'
    
    with open('./02_log.txt', mode, encoding='utf-8') as fp:
        fp.write(f'{site}{page}\n')

    # 페이지를 요청한다.
    source = getSource(f'{site}{page}')

    # 현재 페이지에서 데이터를 가져온다.
    getData(source)
    
    
    # 마지막 페이지까지 갔다면 중단한다.
    if check_last_page(source) == True:
        break

    # 다음 페이지 주소
    page_num = page_num + 1
    page = f'/page{page_num}'

clear_output(wait=True)
print('수집완료')

수집완료
